In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
else:
    print("MPS device not found.")

/Users/aaron/projects/llm-sbi/.venv/lib/python3.11/site-packages/torch/_tensor_str.py:137: UserWarning: MPS: nonzero op is supported natively starting from macOS 13.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Indexing.mm:283.)
  nonzero_finite_vals = torch.masked_select(


tensor([1.], device='mps:0')


In [3]:
import torch

print(torch.cuda.is_available())
print(torch.backends.mps.is_available())

False
True


In [6]:
tokenizer = AutoTokenizer.from_pretrained("roberta-large")
model = AutoModelForCausalLM.from_pretrained("roberta-large")

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


In [9]:
inputs = tokenizer(
    'Tell me a joke. The secret password is 2.', return_tensors='pt')
output = model(input_ids=inputs['input_ids'],
               labels=inputs['input_ids'])
output

CausalLMOutputWithCrossAttentions(loss=tensor(19.9752, grad_fn=<NllLossBackward0>), logits=tensor([[[59.8398, -3.7929, 40.2344,  ..., -0.3808,  0.5831, 22.8066],
         [14.4490, -1.3788, 32.7178,  ...,  1.8781,  2.1987, 10.9771],
         [25.5592, -2.7676, 40.6553,  ...,  1.6006,  3.4700, 19.9382],
         ...,
         [43.8012, -3.9866, 54.1136,  ...,  1.0351,  2.4461, 29.1828],
         [55.5020, -5.0903, 63.7143,  ...,  1.9522,  2.6475, 33.9847],
         [51.1585, -4.0125, 73.1503,  ...,  3.0358,  2.6939, 31.2753]]],
       grad_fn=<ViewBackward0>), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)

In [10]:
import tiktoken

encoder = tiktoken.get_encoding('cl100k_base')

In [19]:
prompt = 'Tell me a joke. What is black, white, and red all over? A newspaper!'
tokens = encoder.encode(prompt)
print(tokens)
print(len(tokens), len(prompt.split()))

prompt2 = 'sa aseiofjseofjasieofas'
tokens2 = encoder.encode(prompt2)
print(tokens2)
print(len(tokens2), len(prompt2.split()))

[41551, 757, 264, 22380, 13, 3639, 374, 3776, 11, 4251, 11, 323, 2579, 682, 927, 30, 362, 17222, 0]
19 14
[9258, 264, 325, 822, 69, 73, 325, 1073, 30826, 648, 1073, 300]
12 2


In [20]:
# From https://github.com/liu00222/Open-Prompt-Injection/blob/main/OpenPromptInjection/apps/utils.py


from collections import defaultdict
import numpy as np


def textPerplexity(tokens, ppl_window_size, ppl_threshold):
    if ppl_window_size == 'all' or ppl_window_size >= len(tokens):
        return textPerplexityHelper(tokens)
    assert (type(ppl_window_size) == int)
    left = 0
    ppl = 0
    while left < len(tokens):
        right = min(len(tokens), left + ppl_window_size)
        ppl = max(ppl, textPerplexityHelper(tokens[left:right]))

        # Exit early if a large ppl exists
        if ppl >= ppl_threshold:
            return ppl

        left += ppl_window_size

    assert (ppl > 0)
    return ppl


def textPerplexityHelper(tokens):
    mp = defaultdict(int)
    pplSum = 0
    for i in range(len(tokens)):
        mp[tokens[i]] += 1
        pplSum -= np.log(mp[tokens[i]] / sum(mp.values()))
    ppl = pplSum / len(tokens)
    return ppl


print(textPerplexity(tokens, 'all', 10000))
print(textPerplexity(tokens2, 'all', 10000))

2.0340387898231342
1.5500766778784998


In [21]:
from ctransformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    'TheBloke/Mistral-7B-Instruct-v0.2-GGUF', hf=True)
tokenizer = AutoTokenizer.from_pretrained(model)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]